In [4]:
import itertools
from concurrent.futures.thread import ThreadPoolExecutor
import requests
import pandas as pd
from sqlalchemy import MetaData, create_engine
import networkx as nx
import math

# System Data

In [5]:
engine = create_engine('sqlite:///eve.db')
connection = engine.connect()
metadata = MetaData()

In [6]:
def system(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/systems/{id}/?datasource=tranquility&language=en').json()
    systemdict = {}
    systemdict['system_id'] = data['system_id']
    systemdict['name'] = data['name']
    systemdict['constellation_id'] = data['constellation_id']
    systemdict['security_status'] = data['security_status']
    systemdict['x'] = data['position']['x']
    systemdict['y'] = data['position']['y']
    systemdict['z'] = data['position']['z']
    return systemdict

def getsystems(systems):
    with ThreadPoolExecutor() as tp:
        results = tp.map(system,systems)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/systems/?datasource=tranquility')
systems = r.json()
syslist = getsystems(systems)
assert len(syslist) == len(systems)
df = pd.DataFrame(syslist)
df.to_sql('systems', con = connection, index = False, if_exists='replace')

# Constellations

In [7]:
def constellation(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/constellations/{id}/?datasource=tranquility&language=en').json()
    constellationdict = {}
    constellationdict['constellation_id'] = data['constellation_id']
    constellationdict['name'] = data['name']
    constellationdict['region_id'] = data['region_id']
    return constellationdict

def getconstellations(constellations):
    with ThreadPoolExecutor() as tp:
        results = tp.map(constellation,constellations)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/constellations/?datasource=tranquility')
constellations = r.json()
constlist = getconstellations(constellations)
assert len(constlist) == len(constellations)
df = pd.DataFrame(constlist)
df.to_sql('constellations', con = connection, index = False, if_exists='replace')

# Regions

In [8]:
def region(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/regions/{id}/?datasource=tranquility&language=en').json()
    regiondict = {}
    regiondict['region_id'] = data['region_id']
    regiondict['name'] = data['name']
    return regiondict

def getregions(regions):
    with ThreadPoolExecutor() as tp:
        results = tp.map(region,regions)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/regions/?datasource=tranquility')
regions = r.json()
reglist = getregions(regions)
assert len(reglist) == len(regions)
df = pd.DataFrame(reglist)
df.to_sql('regions', con = connection, index = False, if_exists='replace')

# System Kills

In [9]:
data = requests.get('https://esi.evetech.net/latest/universe/system_kills/?datasource=tranquility').json()
df = pd.DataFrame(data)
df = df.reindex(columns = ['system_id','npc_kills','ship_kills','pod_kills'])
df.to_sql('system_kills', con = connection, index= False, if_exists = 'replace')

# Thera Connections

In [10]:
data = requests.get('https://www.eve-scout.com/api/wormholes').json()
theradata = []
for tdict in data:
    theradata.append({
        'system_id':tdict['destinationSolarSystem']['id'],
        'type':tdict['sourceWormholeType']['name']
     })
df = pd.DataFrame(theradata)
df.to_sql('thera', con = connection, index = False, if_exists='replace')

# Stargates

In [11]:
def stargate(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/systems/{id}/?datasource=tranquility&language=en').json()
    try:
        return data['stargates']
    except KeyError:
        return []

def getstargates(stargates):
    with ThreadPoolExecutor() as tp:
        results = tp.map(stargate,stargates)
        rlist = [result for result in results]
        return list(itertools.chain(*rlist))


r = requests.get('https://esi.evetech.net/latest/universe/systems/?datasource=tranquility')
stargates = r.json()
gatelist = getstargates(stargates)
df = pd.DataFrame(gatelist)
df.columns = ['gate_id']
df.to_sql('stargates', con = connection, index = False, if_exists='replace')

# Connections


In [12]:
def sconnection(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/stargates/{id}/?datasource=tranquility').json()
    return {
        'source':data['system_id'],
        'destination':data['destination']['system_id']
    }

def getconnections(stargates):
    with ThreadPoolExecutor() as tp:
        results = tp.map(sconnection,stargates)
        return [result for result in results]


stargates = connection.execute('select gate_id from stargates').fetchall()
stargates = [y for x in stargates for y in x]
conlist = getconnections(stargates)
assert len(conlist) == len(stargates)
df = pd.DataFrame(conlist)
df.to_sql('connections', con = connection, index = False, if_exists='replace')

In [14]:
noduplicates = connection.execute('SELECT DISTINCT source, destination FROM connections t1 WHERE t1.source > t1.destination OR NOT EXISTS (SELECT * FROM connections t2 WHERE t2.source = t1.destination AND t2.destination = t1.source)').fetchall()
df = pd.DataFrame(noduplicates)
df.columns = ['source_id','destination_id']
df.to_sql('connections',con = connection, index=False, if_exists='replace')

In [15]:
eve = nx.Graph()
eve.add_edges_from(noduplicates)

In [16]:
query = """
SELECT s.system_id, s.name as system, security_status, r.name as region, npc_kills, pod_kills, ship_kills, x, y, z
FROM systems AS s
         INNER JOIN constellations c ON c.constellation_id = s.constellation_id
         INNER JOIN regions r ON r.region_id = c.region_id
         INNER JOIN system_kills sk ON s.system_id = sk.system_id
         """
test = connection.execute(query)
df = pd.DataFrame(test, columns=test.keys()).sort_values(by=['npc_kills'], ascending=False)
df

,system_id,system,security_status,region,npc_kills,pod_kills,ship_kills,x,y,z
1901,30003645,O36A-P,-0.084704,Tenal,1460,0,0,-6.397321e+16,8.255295e+16,4.294112e+17
2216,30004142,Hikansog,0.811302,Kador,1371,0,0,-2.352238e+17,5.373238e+16,-1.218548e+17
657,30001407,Todaki,1.000000,Lonetrek,927,0,0,-1.738105e+17,9.508494e+16,1.223923e+17
383,30000897,F-G7BO,-0.593187,Tribute,903,0,0,-5.445899e+16,7.331115e+16,2.359767e+17
2589,30005001,Arnon,0.593093,Essence,890,0,3,-2.373953e+17,5.459030e+16,3.017365e+16
...,...,...,...,...,...,...,...,...,...,...
2533,30004916,H90-C9,-0.532198,Omist,0,1,0,1.288425e+17,-2.304870e+16,-4.117844e+17
1025,30002310,YX-0KH,-0.030253,Etherium Reach,0,0,1,1.854030e+17,8.149192e+16,3.458675e+16
2122,30003982,K-L690,-0.046148,Querious,0,1,1,-3.487798e+17,2.896199e+16,-2.406486e+17
2454,30004716,4K-TRB,-0.020233,Delve,0,0,1,-4.242369e+17,2.986694e+16,-2.173928e+17


In [17]:
query = """
SELECT thera.system_id, name, x, y, z from thera
INNER JOIN systems s ON s.system_id = thera.system_id
"""
theradata = connection.execute(query)
thera = pd.DataFrame(theradata, columns= theradata.keys())
thera

,system_id,name,x,y,z
0,30004658,ZUE-NS,-5.004733e+17,4.877824e+16,-5.277791e+16
1,30002096,Hofjaldgund,-1.320439e+17,1.900875e+16,1.459811e+16
2,31001558,J124727,7.478041e+18,-6.819950e+15,-9.579101e+18
3,30000937,ZJ-GOU,5.171652e+16,3.615544e+15,-1.047533e+17
4,30002752,Ahynada,-1.318148e+17,8.897007e+16,1.318486e+17
5,30004977,Yvangier,-2.088278e+17,5.357265e+16,7.707838e+16
6,30002217,Hutian,-1.994459e+17,6.876697e+16,-1.047576e+17
7,31000402,J233534,7.783411e+18,-1.695121e+16,-9.349452e+18
8,30004818,C-FD0D,8.115914e+15,1.349033e+16,-2.665072e+17
9,30004257,Hakatiz,-3.636554e+17,3.879064e+16,-4.301449e+16


In [18]:
closest_gate = []
for row in df.itertuples():
    shortest = int(10000000)
    closest = int(0)
    for trow in thera.itertuples():
        try:
            pathlength = len(nx.shortest_path(eve,source = row.system_id, target = trow.system_id))
            if shortest > pathlength:
                shortest = pathlength
                closest = trow.name
        except:
            continue
    closest_gate.append(
        {
            'closest_gate': closest,
            'gate_distance': shortest
        }
    )

In [19]:
len(closest_gate)
df['closest_gate'] = [x['closest_gate'] for x in closest_gate]
df['gate_distance'] = [x['gate_distance'] for x in closest_gate]
df

,system_id,system,security_status,region,npc_kills,pod_kills,ship_kills,x,y,z,closest_gate,gate_distance
1901,30003645,O36A-P,-0.084704,Tenal,1460,0,0,-6.397321e+16,8.255295e+16,4.294112e+17,S-1ZXZ,15
2216,30004142,Hikansog,0.811302,Kador,1371,0,0,-2.352238e+17,5.373238e+16,-1.218548e+17,Kooreng,9
657,30001407,Todaki,1.000000,Lonetrek,927,0,0,-1.738105e+17,9.508494e+16,1.223923e+17,Ahynada,7
383,30000897,F-G7BO,-0.593187,Tribute,903,0,0,-5.445899e+16,7.331115e+16,2.359767e+17,S-1ZXZ,9
2589,30005001,Arnon,0.593093,Essence,890,0,3,-2.373953e+17,5.459030e+16,3.017365e+16,Yvangier,7
...,...,...,...,...,...,...,...,...,...,...,...,...
2533,30004916,H90-C9,-0.532198,Omist,0,1,0,1.288425e+17,-2.304870e+16,-4.117844e+17,C-FD0D,29
1025,30002310,YX-0KH,-0.030253,Etherium Reach,0,0,1,1.854030e+17,8.149192e+16,3.458675e+16,LEM-I1,17
2122,30003982,K-L690,-0.046148,Querious,0,1,1,-3.487798e+17,2.896199e+16,-2.406486e+17,C-FD0D,26
2454,30004716,4K-TRB,-0.020233,Delve,0,0,1,-4.242369e+17,2.986694e+16,-2.173928e+17,ZUE-NS,16


In [20]:
def distance(curloc, otherloc):
    dist = math.sqrt(
        (curloc[0] - otherloc[0]) ** 2 + (curloc[1] - otherloc[1]) ** 2 + (curloc[2] - otherloc[2]) ** 2)
    return dist / 149597870691 / 63239.6717


def closestly(indf,thera):
    dists = []
    names = []
    for tup in indf.itertuples():
        tempdist = []
        tempname = []
        for tup2 in thera.itertuples():
            tempdist.append(distance((tup.x, tup.y, tup.z), (tup2.x, tup2.y, tup2.z)))
            tempname.append(tup2.name)
        smallest = tempdist.index(min(tempdist))
        dists.append(tempdist[smallest])
        names.append(tempname[smallest])

    return names, dists

In [21]:
df['closest_ly'], df['distance_ly'] = closestly(df,thera)
df[['system','region','npc_kills','pod_kills','ship_kills','closest_gate','gate_distance','closest_ly','distance_ly']]

,system,region,npc_kills,pod_kills,ship_kills,closest_gate,gate_distance,closest_ly,distance_ly
1901,O36A-P,Tenal,1460,0,0,S-1ZXZ,15,S-1ZXZ,11.995180
2216,Hikansog,Kador,1371,0,0,Kooreng,9,Kooreng,4.323074
657,Todaki,Lonetrek,927,0,0,Ahynada,7,Ahynada,4.595873
383,F-G7BO,Tribute,903,0,0,S-1ZXZ,9,Eruka,10.727274
2589,Arnon,Essence,890,0,3,Yvangier,7,Fahruni,5.701253
...,...,...,...,...,...,...,...,...,...
2533,H90-C9,Omist,0,1,0,C-FD0D,29,C-FD0D,20.336515
1025,YX-0KH,Etherium Reach,0,0,1,LEM-I1,17,LEM-I1,9.543058
2122,K-L690,Querious,0,1,1,C-FD0D,26,Nakregde,17.582562
2454,4K-TRB,Delve,0,0,1,ZUE-NS,16,ZUE-NS,19.279523
